In [38]:
import pandas as pd
import numpy as np
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from sklearn.preprocessing import MultiLabelBinarizer

In [29]:
df = pd.read_csv('preprocessed_data_Jul14.csv')
df

,concept_id,work_id,publication_year,title,paperabstract,country,year_concept,concatenated_title_abstract,US,IN,...,pub_interval_2015_2019,pub_interval_2010_2014,pub_interval_2005_2009,pub_interval_2000_2004,pub_interval_1995_1999,pub_interval_1985_1994,pub_interval_1975_1984,pub_interval_1965_1974,pub_interval_1900_1964,pub_interval_1824_1899
0,https://openalex.org/C44870925,https://openalex.org/W1993867637,2004,KINEMATIC TREATMENT OF CORONAL MASS EJECTION E...,We present a kinematic study of the evolution ...,US US US US,2004+https://openalex.org/C44870925,KINEMATIC TREATMENT OF CORONAL MASS EJECTION E...,100.000000,0.0,...,0,0,0,1,0,0,0,0,0,0
1,https://openalex.org/C44870925,https://openalex.org/W2022503540,1991,The imaging performance of the Hubble Space Te...,Problems with the HST instantaneous imaging pe...,US,1991+https://openalex.org/C44870925,The imaging performance of the Hubble Space Te...,100.000000,0.0,...,0,0,0,0,0,1,0,0,0,0
2,https://openalex.org/C44870925,https://openalex.org/W2077626606,2003,Numerical Simulations of Mass Outflows Driven ...,"We study the two-dimensional, time-dependent m...",US,2003+https://openalex.org/C44870925,Numerical Simulations of Mass Outflows Driven ...,100.000000,0.0,...,0,0,0,1,0,0,0,0,0,0
3,https://openalex.org/C1276947,https://openalex.org/W3104346433,2003,The Origin of X-shaped Radio Galaxies: Clues f...,Existing radio images of a few X-shaped radio ...,IN US,2003+https://openalex.org/C1276947,The Origin of X-shaped Radio Galaxies: Clues f...,50.000000,50.0,...,0,0,0,1,0,0,0,0,0,0
4,https://openalex.org/C44870925,https://openalex.org/W2081151530,2002,The Ghost of Sagittarius and Lumps in the Halo...,We identify new structures in the halo of the ...,US US US US,2002+https://openalex.org/C44870925,The Ghost of Sagittarius and Lumps in the Halo...,100.000000,0.0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63994,https://openalex.org/C44870925,https://openalex.org/W3102168087,2003,Accretion Disk Spectra of the Ultra-luminous X...,Ultra-luminous Compact X-ray Sources (ULXs) in...,US CH JP,2003+https://openalex.org/C44870925,Accretion Disk Spectra of the Ultra-luminous X...,33.333333,0.0,...,0,0,0,1,0,0,0,0,0,0
63995,https://openalex.org/C1276947,https://openalex.org/W3098886327,2008,The ghost of a dwarf galaxy: fossils of the hi...,We present an extragalactic perspective of an ...,GB US,2008+https://openalex.org/C1276947,The ghost of a dwarf galaxy: fossils of the hi...,50.000000,0.0,...,0,0,1,0,0,0,0,0,0,0
63996,https://openalex.org/C44870925,https://openalex.org/W3093754290,2020,Observations of the Solar Corona from Space,Space observations of the atmosphere of the Su...,IT IT,2020+https://openalex.org/C44870925,Observations of the Solar Corona from Space Sp...,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
63997,https://openalex.org/C44870925,https://openalex.org/W3121322575,2011,Constraints on the topology of the Universe de...,We impose constraints on the topology of the U...,CH CH,2011+https://openalex.org/C44870925,Constraints on the topology of the Universe de...,0.000000,0.0,...,0,1,0,0,0,0,0,0,0,0


In [30]:
# Extracting the "concatenated_title_abstract" column
texts = df.iloc[:, 7].values 
texts

array(['KINEMATIC TREATMENT OF CORONAL MASS EJECTION EVOLUTION IN THE SOLAR WIND We present a kinematic study of the evolution of coronal mass ejections (CMEs) in the solar wind. Specifically, we consider the effects of (1) spherical expansion and (2) uniform expansion due to pressure gradients between the interplanetary CME (ICME) and the ambient solar wind. We compare these results with an MHD model that allows us to isolate these effects from the combined kinematic and dynamical effects, which are included in MHD models. They also provide compelling evidence that the fundamental cross section of so-called "force-free" flux ropes (or magnetic clouds) is neither circular or elliptical, but rather a convex-outward, "pancake" shape. We apply a force-free fit to the magnetic vectors from the MHD simulation to assess how the distortion of the flux rope affects the fit. In spite of these limitations, force-free fits, which are straightforward to apply, do provide an important description o

In [31]:
stop_words = set(stopwords.words('english'))

In [32]:
# 1. Text cleaning
def clean_text(text):
    text = re.sub(r"[^a-zA-Z\s]", "", text)  # Remove non-alphabetic characters
    text = text.lower()  # Convert text to lowercase
 # Remove stop words
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text.strip()

cleaned_texts = [clean_text(text) for text in texts]
cleaned_texts


['kinematic treatment coronal mass ejection evolution solar wind present kinematic study evolution coronal mass ejections cmes solar wind specifically consider effects spherical expansion uniform expansion due pressure gradients interplanetary cme icme ambient solar wind compare results mhd model allows us isolate effects combined kinematic dynamical effects included mhd models also provide compelling evidence fundamental cross section socalled forcefree flux ropes magnetic clouds neither circular elliptical rather convexoutward pancake shape apply forcefree fit magnetic vectors mhd simulation assess distortion flux rope affects fit spite limitations forcefree fits straightforward apply provide important description number parameters including radial dimension orientation chirality icme',
 'imaging performance hubble space telescope problems hst instantaneous imaging performance pointing performance discussed optical tests clearly demonstrated hst suffers spherical aberration top level

In [33]:
# 2. Tokenization
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(cleaned_texts)
sequences = tokenizer.texts_to_sequences(cleaned_texts)
sequences


[[923,
  1753,
  320,
  4,
  1186,
  28,
  32,
  160,
  20,
  923,
  83,
  28,
  320,
  4,
  1725,
  1203,
  32,
  160,
  1909,
  639,
  135,
  942,
  631,
  1285,
  631,
  113,
  503,
  1078,
  1607,
  1289,
  5447,
  1412,
  32,
  160,
  419,
  23,
  1229,
  14,
  651,
  333,
  4955,
  135,
  490,
  923,
  275,
  135,
  1275,
  1229,
  17,
  16,
  176,
  3716,
  95,
  669,
  1747,
  2193,
  1793,
  3961,
  67,
  3976,
  36,
  192,
  2375,
  931,
  461,
  499,
  1,
  1,
  458,
  1137,
  3961,
  467,
  36,
  3953,
  1229,
  655,
  1953,
  2623,
  67,
  2340,
  2213,
  467,
  3717,
  2512,
  3961,
  851,
  3741,
  1137,
  176,
  259,
  1501,
  116,
  103,
  224,
  191,
  4141,
  1423,
  9376,
  5447],
 [210,
  2028,
  213,
  96,
  79,
  1572,
  664,
  3438,
  210,
  2028,
  2971,
  2028,
  533,
  42,
  1328,
  895,
  2174,
  664,
  6199,
  942,
  7129,
  2731,
  362,
  1,
  1577,
  35,
  3126,
  131,
  20,
  263,
  7734,
  1294,
  3759,
  1110,
  1577,
  2971,
  2138,
  65,
  16,
  1572

In [34]:
# 3. Padding sequences
padded_sequences = pad_sequences(sequences, padding='post', maxlen=300)
padded_sequences


array([[ 923, 1753,  320, ...,    0,    0,    0],
       [ 210, 2028,  213, ...,    0,    0,    0],
       [ 378,   91,    4, ...,    0,    0,    0],
       ...,
       [   8,   32,  722, ...,    0,    0,    0],
       [ 281, 3176,  158, ...,    0,    0,    0],
       [  10, 8739,  196, ...,    0,    0,    0]], dtype=int32)

In [36]:
labels_df = pd.read_csv('Labels.csv')

In [39]:
# 4. Binarize labels for multi-label classification
mlb = MultiLabelBinarizer()
binarized_labels = mlb.fit_transform(labels_df.values)
binarized_labels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [40]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, binarized_labels, test_size=0.2, random_state=42)


ValueError: Found input variables with inconsistent numbers of samples: [63999, 1042]

In [41]:
# 5. Create the model
vocab_size = len(tokenizer.word_index) + 1
num_labels = binarized_labels.shape[1]

model = Sequential([
    Embedding(vocab_size, 16, input_length=300),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_labels, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 300, 16)           1311504   
                                                                 
 flatten_2 (Flatten)         (None, 4800)              0         
                                                                 
 dense_4 (Dense)             (None, 128)               614528    
                                                                 
 dense_5 (Dense)             (None, 995)               128355    
                                                                 
Total params: 2054387 (7.84 MB)
Trainable params: 2054387 (7.84 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [42]:
#Prediction
# 1. Get the probabilities
probabilities = model.predict(padded_sequences)

# 2. Convert probabilities to label predictions using a threshold
threshold = 0.5
predictions = (probabilities > threshold).astype(int)

# 3. Convert binary predictions back to label names
predicted_labels = mlb.inverse_transform(predictions)

# Display predictions for the first few abstracts
for i, labels in enumerate(predicted_labels[:10]):
    print(f"Abstract {i + 1}: {', '.join(labels)}")


2000/2000 [==============================] - 3s 1ms/step
Abstract 1: 10.5 inch, McMath-Hulbert Observatory, 20 m telescope, 32 m VLBI dish, 45-cm-Turmteleskop, 50-foot tower, McMath-Hulbert Observatory, ALLBIN (Amateur Linked Long Baseline Interferometer Network), AMiBA, ANTARES, ARIES Observatory, ARO 12m Radio Telescope, Abastumani Astrophysical Observatory, Abu Reyhan-e Birooni Observatory, Adirondack Public Observatory, Adolphson Observatory, Airdrie Public Observatory, Akari (Astro-F), Aker Observatory, Algonquin Radio Observatory, Allegheny Observatory, Allen Telescope Array, Alpha Magnetic Spectrometer 01 (AMS-01), Alpha Magnetic Spectrometer 02 (AMS-02), Anderson Mesa Station, Ankara University Observatory, Antarctic Submillimeter Telescope and Remote Observatory, Apache Point Observatory, Apollo Observatory, Arcetri Observatory, Arecibo Telescope, Argentine Institute of Radio Astronomy, Armagh Observatory, Ashton Observatory, Assheton Observatory, Astron, Astronomical Observat